In [ ]:
# Library Imports
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, set_seed
from datasets import *
import numpy as np
import pandas as pd
import torch
import ast
import evaluate

In [ ]:
# Set the seed
SEED = 42
set_seed(SEED)

# Standard model path
local_models_path = '../models/T5'

# Boilerplate results
results = pd.DataFrame(columns=['Model', 'Train Data', 'Data Type', 'Bert.Precision', 'Bert.Recall', 'Bert.F1', 'BLEU'])

In [ ]:
# Define the prediction function
def get_answer(question, context, tokenizer, model):
    input_text = "question: %s  context: %s" % (question, context)
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], attention_mask=features['attention_mask'])

    return tokenizer.decode(output[0], skip_special_tokens=True)

## Load all data

2020

In [ ]:
# Load the dataset from file and split it into train and test datasets
data_2020_full = load_dataset('csv', data_files=f"../data/clean/squad/sustainability-report-2020-squad-format.csv",
                    delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2020_full["test"] = data_2020_full["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2020_full["test"] = data_2020_full["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# Replace all "\n" with " " in the context, answers and questions
data_2020_full["test"] = data_2020_full["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2020_full["test"] = data_2020_full["test"].remove_columns(["text", "answer_start"])
# Get ground truth answers
test_data_2020_full = data_2020_full["test"]
gt_answers_2020_full = [temp["answers"]["text"][0] for temp in test_data_2020_full]

2022

In [ ]:
data_2022_full = load_dataset('csv', data_files=f"../data/clean/squad/sustainability-report-2022-squad-format.csv",
                    delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2022_full["test"] = data_2022_full["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2022_full["test"] = data_2022_full["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# Replace all "\n" with " " in the context, answers and questions
data_2022_full["test"] = data_2022_full["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2022_full["test"] = data_2022_full["test"].remove_columns(["text", "answer_start"])

test_data_2022_full = data_2022_full["test"]
gt_answers_2022_full = [temp["answers"]["text"][0] for temp in test_data_2022_full]

2020 + 2022

In [ ]:
data_2020_2022 = load_dataset('csv', data_files="../data/clean/squad/sustainability-report-2042-squad-format.csv",
                                delimiter=";", split="train").train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2020_2022["test"] = data_2020_2022["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2020_2022["test"] = data_2020_2022["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# Replace all "\n" with " " in the context, answers and questions
data_2020_2022["test"] = data_2020_2022["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2020_2022["test"] = data_2020_2022["test"].remove_columns(["text", "answer_start"])
# Get ground truth answers
test_data_2020_2022 = data_2020_2022["test"]
gt_answers_2020_2022 = [temp["answers"]["text"][0] for temp in test_data_2020_2022]

2022 handwritten

In [ ]:
data_2022_handwritten = load_dataset('csv', data_files=f"../data/clean/squad/QA_SR_2022_Expert-squad-format.csv",
                                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2022_handwritten["test"] = data_2022_handwritten["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2022_handwritten["test"] = data_2022_handwritten["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# Replace all "\n" with " " in the context, answers and questions
data_2022_handwritten["test"] = data_2022_handwritten["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2022_handwritten["test"] = data_2022_handwritten["test"].remove_columns(["text", "answer_start"])

test_data_2022_handwritten = data_2022_handwritten["test"]
gt_answers_2022_handwritten = [temp["answers"]["text"][0] for temp in test_data_2022_handwritten]

## SIMPLE EVALUATION

In [ ]:
bertscore = evaluate.load("bertscore")
bleu = evaluate.load("bleu")

### Small

In [ ]:
model_name = "mrm8488/t5-small-finetuned-squadv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

answers_2020 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]
answers_2022_handwritten = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2022_handwritten["question"], test_data_2022_handwritten["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)

# add results to dataframe
results.loc[len(results)] = ['t5-small', None, '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-small', None, '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-small', None, '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]
results.loc[len(results)] = ['t5-small', None, '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu']]

### Small - finetuned

In [ ]:
model_name_2020 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2020-full"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020, local_files_only=True)
model_2020 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020, local_files_only=True)

model_name_2022 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-full"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022, local_files_only=True)
model_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022, local_files_only=True)

model_name_2020_2022 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022, local_files_only=True)
model_2020_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020_2022, local_files_only=True)

model_name_2022_handwritten = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten"
tokenizer_2022_handwritten = AutoTokenizer.from_pretrained(model_name_2022_handwritten, local_files_only=True)
model_2022_handwritten = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022_handwritten, local_files_only=True)

answers_2020 = [get_answer(question, context, tokenizer_2020, model_2020) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer_2022, model_2022) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer_2020_2022, model_2020_2022) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]
answers_2022_handwritten = [get_answer(question, context, tokenizer_2022_handwritten, model_2022_handwritten) for question, context in zip(test_data_2022_handwritten["question"], test_data_2022_handwritten["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)

# add results to dataframe
results.loc[len(results)] = ['t5-small', 'full', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-small', 'full', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-small', 'full', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]
results.loc[len(results)] = ['t5-small', 'full', '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu']]

### Small - finetuned - train set halved

In [ ]:
model_name_2020 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2020-smaller"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020, local_files_only=True)
model_2020 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020, local_files_only=True)

model_name_2022 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-smaller"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022, local_files_only=True)
model_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022, local_files_only=True)

model_name_2020_2022 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-smaller_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022, local_files_only=True)
model_2020_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020_2022, local_files_only=True)

answers_2020 = [get_answer(question, context, tokenizer_2020, model_2020) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer_2022, model_2022) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer_2020_2022, model_2020_2022) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)

# add results to dataframe
results.loc[len(results)] = ['t5-small', 'smaller', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-small', 'smaller', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-small', 'smaller', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]

### Base

In [ ]:
model_name = "mrm8488/t5-base-finetuned-squadv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

answers_2020 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]
answers_2022_handwritten = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2022_handwritten["question"], test_data_2022_handwritten["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)

# add results to dataframe
results.loc[len(results)] = ['t5-base', None, '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-base', None, '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-base', None, '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]
results.loc[len(results)] = ['t5-base', None, '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu']]

### Base - finetuned

In [ ]:
model_name_2020 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2020-full"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020, local_files_only=True)
model_2020 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020, local_files_only=True)

model_name_2022 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2022-full"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022, local_files_only=True)
model_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022, local_files_only=True)

model_name_2020_2022 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022, local_files_only=True)
model_2020_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020_2022, local_files_only=True)

model_name_2022_handwritten = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten"
tokenizer_2022_handwritten = AutoTokenizer.from_pretrained(model_name_2022_handwritten, local_files_only=True)
model_2022_handwritten = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022_handwritten, local_files_only=True)

answers_2020 = [get_answer(question, context, tokenizer_2020, model_2020) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer_2022, model_2022) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer_2020_2022, model_2020_2022) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]
answers_2022_handwritten = [get_answer(question, context, tokenizer_2022_handwritten, model_2022_handwritten) for question, context in zip(test_data_2022_handwritten["question"], test_data_2022_handwritten["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)

# add results to dataframe
results.loc[len(results)] = ['t5-base', 'full', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-base', 'full', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-base', 'full', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]
results.loc[len(results)] = ['t5-base', 'full', '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu']]

### Base - finetuned - train set halved

In [ ]:
model_name_2020 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2020-smaller"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020, local_files_only=True)
model_2020 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020, local_files_only=True)

model_name_2022 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2022-smaller"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022, local_files_only=True)
model_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022, local_files_only=True)

model_name_2020_2022 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2042-smaller_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022, local_files_only=True)
model_2020_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020_2022, local_files_only=True)

answers_2020 = [get_answer(question, context, tokenizer_2020, model_2020) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer_2022, model_2022) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer_2020_2022, model_2020_2022) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)

# add results to dataframe
results.loc[len(results)] = ['t5-base', 'smaller', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-base', 'smaller', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-base', 'smaller', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]

## Display all results

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(results)